Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# An end to end example using PyTorch Geometric on IPUs

Graph Neural Networks (GNNs) are models designed to derive insights from unstructured data that can be represented as graphs. 
They are used to address a wide variety of scientific and industrial problems: their application to chemistry and biology has unlocked
a new era of drug discovery, while their use to analyse product and customer interactions is improving recommender systems.

GNNs are ideal to run on Graphcore IPUs, due to IPU architectural characteristics such as very large amounts of on-chip SRAM,
which makes message passing operations typical of GNN workloads much faster than other processor types. For more details about why IPUs 
are extremely capable at running GNN workloads please check out our [blog post](https://www.graphcore.ai/posts/what-gnns-are-great-at-and-why-graphcore-ipus-are-great-at-gnns) on this topic. 
The advantages of running GNNs on IPUs compared to other processor types can be seen in our [benchmarks](../benchmarks).

This tutorial will get you started training and running inference on your first GNN model on IPUs.
We will go through the following steps required to run a PyTorch Geometric model on IPU, detailing the journey of running GNNs on the IPU:
 * Loading a PyTorch Geometric dataset
 * Using a [PopTorch Geometric](https://docs.graphcore.ai/projects/poptorch-geometric-user-guide/) dataloader to achieve fixed size outputs
 * Creating a model
 * Adjusting the model to satisfy PopTorch requirements
 * Start training on the IPU
 * Run inference on the IPU

The specific task we will cover is to classify the nodes of the PyTorch Geometric [Cora citation network dataset](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.Planetoid);

While this tutorial will cover enough of the basics of GNNs, PyTorch Geometric and PopTorch
for you to start developing and porting your GNN applications to the IPU;
the following resources can be used to complement your understanding of:

- PopTorch : [Introduction to PopTorch - running a simple model](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/basics);
- GNNs : [A Gentle Introduction to Graph Neural Networks](https://distill.pub/2021/gnn-intro/)
- PyTorch Geometric (PyG): [Official notebooks examples and tutorials](https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html)

## Running on Paperspace

The Paperspace environment lets you run this notebook with no set up. To improve your experience we preload datasets and pre-install packages, this can take a few minutes, if you experience errors immediately after starting a session please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to through our community of developers using our [slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/examples).

Requirements:

* Python packages installed with `pip install -r ../requirements.txt`

In [ ]:
%pip install -q -r ../requirements.txt

And for compatibility with the Paperspace environment variables we will do the following:

In [ ]:
import os

executable_cache_dir = (
    os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/")
    + "/pyg-a-worked-example"
)
dataset_directory = os.getenv("DATASET_DIR", "data")

Now we are ready to start!

## Loading a PyTorch Geometric dataset

PyTorch Geometric provides simple access to many of the datasets used in literature for GNNs.
The full list of datasets is available in the [project's documentation](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch-geometric-datasets).

For this tutorial we will use the `Cora` citation network dataset within the `Planetoid` benchmark suite to train our first GNN and run it on the IPU for the task of node classification.
The `Cora` dataset features a single graph where nodes represent documents and edges represent citation links. Let's load the dataset and take a look at some of its features.

In [ ]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

transform = T.Compose([T.NormalizeFeatures(), T.AddSelfLoops()])

dataset = Planetoid(root=dataset_directory, name="Cora", transform=transform)
data = dataset[0]  # Access the citation graph as Data object

print(f"Dataset: {dataset}: ")
print(f"Number of graphs: {len(dataset)}: ")
print(f"Number of features: {dataset.num_features}: ")
print(f"Number of classes: {dataset.num_classes}: ")

print(data)

print(f"{data.num_nodes = }")
print(f"{data.num_edges = }")

Graphs in PyTorch Geometric are stored in `Data` objects which provide an expressive string representation (see above), and a neat interface for accessing properties of the graph, for example its number of nodes and edges.
The `edge_index` property describes the graph connectivity, while `x` representes the node features (1433-dim feature vector for each node) and `y` represents the node labels (each node is associated to one class).
It's also worth noting the mask properties which denotes against which nodes to train/validate/test as we already know their community assignment. 
PyTorch Geometric datasets support transforms for pre-processing datasets on CPU before passing them to the model. In addition to using transforms for data augmentation they can also be useful to apply data processing which does not have learnable parameters and is not supported on the IPU.

You may notice that in the code block above we made use of two transforms:
- `NormalizeFeatures()` to row-normalize the input feature vector;
- `AddSelfLoops()` to add self-loops to the dataset. Self-loops are edges which connect a node to itself and facilitate the communication of the node's feature across layers. Implementing self-loops corresponds to filling in the diagonal of the adjacency matrix.

Now we have the dataset loaded we can use a dataloader to efficiently load our data onto the IPU, in the next section we will see how to do that.

## Using a dataloader

There are a number of reasons why you may want to use a dataloader to load your dataset into your model. When you are using IPUs, the main reasons to use a dataloader are:
 * to achieve the most efficient loading performance;
 * to enable using particular PopTorch features such as replication and gradient accumulation;
 * to achieve fixed size batches required for the IPU.

[PopTorch Geometric](https://docs.graphcore.ai/projects/poptorch-geometric-user-guide/), the IPU-specific PyTorch Geometric library, provides a wrapper for the PopTorch dataloader, making it easy to get performant PyTorch Geometric models running on the IPU. As we only have a single item in our dataset we do not need to worry about making our batches fixed size so we can use the most basic of PopTorch Geometric's dataloaders.

In [ ]:
from poptorch_geometric.dataloader import DataLoader

dataloader = DataLoader(dataset, batch_size=1)

Our dataloader is now ready to go. Let's move on to creating a model.

## Creating a model

Let's remind ourselves of the task to inform the model we want to create. We want to classify the nodes of the Cora citation network dataset. Let's take a look at some important GNN features to help us construct our model.

### Message passing layers in PyTorch Geometric

GNNs rely on message passing schemes to perform neighbourhood aggregation of node and edge features, explained in more detail in [PyTorch Geometric white paper](https://arxiv.org/abs/1903.02428).
This scheme aggregates features along the structure of the graph and it calculates updated feature vectors for each node and edge.
MLP Layers with learnable parameters can be introduced in the calculation:

- During the message passing: features of a neighbouring node and a connecting edge may be processed through a dense layer to calculate a message;
- After aggregation: the combined messages are processed through neural layers to learn new derived features.

An important property of neighbourhood aggregation schemes is that they are "permutation invariant": the order in which nodes or edges appear in the arrays does not affect them.

PyTorch Geometric offers a large number of message passing layers which aggregate node and edge features. The full list is available in the [convolutional layers](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers) section of the PyTorch Geometric documentation.

Let's take a look at the support convolutional layers:

In [ ]:
from torch_geometric import nn

# List all the layers which are a subclass of the MessagePassing layer
attrs = []
for attr in dir(nn):
    try:
        if issubclass(getattr(nn, attr), nn.MessagePassing):
            attrs.append(attr)
    except:
        pass
print(attrs)

For our task, a simple `GCNConv` layer will suffice. As we have added the self-loops to the dataset as a transform we can turn them off in the layer. Turning off the self-loops in the layer is also a change we must make to ensure our compiled graph for the IPU is static.

In [ ]:
from torch_geometric.nn import GCNConv

conv = GCNConv(16, 16, add_self_loops=False)

Now we know about message passing, let's construct a model with these layers.

### Using the GCNConv layer in a model

We are now ready to write a small GNN model to process the Cora dataset using the `GCNConv` layer provided by PyTorch Geometric. We do this in the normal way with two small changes:
 * PopTorch requires the loss function to be part of the model, so we will move that in to the end of the forward pass.
 * Set the labels of the masked nodes to `-100` so they are ignored in the loss function.

In [ ]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F


class GCN(torch.nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16, add_self_loops=False)
        self.conv2 = GCNConv(16, out_channels, add_self_loops=False)

    def forward(self, x, edge_index, y=None, train_mask=None):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index).relu()
        x = F.log_softmax(x, dim=1)

        if self.training:
            y = torch.where(train_mask, y, -100)
            loss = F.nll_loss(x, y)
            return x, loss
        return x

The number of in channels for our model is the size of the input features. The number of out channels is the number of classes in our dataset.

In [ ]:
print(f"{dataset.num_node_features = }")
print(f"{dataset.num_classes = }")

Now we have described our model, let's initialise it. 

In [ ]:
in_channels = dataset.num_node_features
out_channels = dataset.num_classes

model = GCN(in_channels, out_channels)
model.train()

Our model is ready for training, we will learn how to do that in the next section.

## Training our model

We are now in the position to begin training our model. To make this model ready for training on the IPU we will wrap the model in PopTorch functionality. We do the following:
 * Use a PopTorch optimizer for better speed and memory performance on the IPU. `Adam` is a suitable optimizer for our task, so we can create the optimizer in the typical way.
 * Wrap the model in `poptorch.trainingModel`;
 * Use PopTorch options to make use of PopTorch and IPU features, here we only set a directory to save and load our compiled executable from. Take a look at the [PopTorch documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/reference.html?highlight=options#poptorch.Options) for some of the other features you could use.

In [ ]:
import poptorch

optimizer = poptorch.optim.Adam(model.parameters(), lr=0.001)
poptorch_options = poptorch.Options().enableExecutableCaching(executable_cache_dir)
poptorch_model = poptorch.trainingModel(model, poptorch_options, optimizer=optimizer)

We can then set up the training loop in a familiar way. The key things to note is, because we moved the loss function inside the model, we do not need to define the loss calculation, the backward pass or the optimizer step. PopTorch handles this all for us.

Let's set up the training loop and run some epochs.

In [ ]:
from tqdm import tqdm

losses = []

for epoch in tqdm(range(100)):
    bar = tqdm(dataloader)
    for data in bar:
        _, loss = poptorch_model(
            data.x, data.edge_index, y=data.y, train_mask=data.train_mask
        )
        bar.set_description(f"Epoch {epoch} loss: {loss:0.6f}")
        losses.append(loss)

Our loss is decreasing nicely! We can now detach our training model from the IPU.

In [ ]:
poptorch_model.detachFromDevice()

And let's visualise the loss per epoch.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(list(range(len(losses))), losses)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
plt.grid(True)

Looks like our model hasn't converged yet, feel free to increase the number of epochs and attempt to reach full convergence.

Now we have trained the model, let's see how it does on our validation and test sets.

## Running inference on the trained model

Once training is complete, it is typical to test its performance on the validation and test datasets, we will see how to do that in this section.

For inference, we wrap set the model to evaluation mode and wrap in `poptorch.inferenceModel`.

In [ ]:
model.eval()
poptorch_inf_model = poptorch.inferenceModel(model, options=poptorch_options)

We can use this model in inference as normal. In the following we get the single large graph of data and feed it into the inference model, get the result, finally detaching our model from the IPU.

In [ ]:
data = next(iter(dataset))
logits = poptorch_inf_model(data.x, data.edge_index)
poptorch_inf_model.detachFromDevice()

We get the logits back with which we can calculate the accuracy of our model. First we will convert our logits to predictions. As this is a classification task with 7 possible classes, we find the position where the logits are the largest, this is the predicted class for that particular node.

In [ ]:
pred = logits.argmax(dim=1)
pred

Now we can calculate how many of these predictions are correct. We use `val_mask` for this to ensure we are only taking the accuracy of the nodes we care about, the ones in the validation dataset.

In [ ]:
correct_results = pred[data.val_mask] == data.y[data.val_mask]
accuracy = int(correct_results.sum()) / int(data.val_mask.sum())
print(f"Validation accuracy: {accuracy:.2%}")

And we do the same with the test dataset, using the `test_mask`.

In [ ]:
correct_results = pred[data.test_mask] == data.y[data.test_mask]
accuracy = int(correct_results.sum()) / int(data.test_mask.sum())
print(f"Test accuracy: {accuracy:.2%}")

Here we did the accuracy calculation on the CPU. We could equally have placed the functionality in our model and let the IPU do the work. Why not try to make a change to return the accuracy from the IPU instead of calculating on the CPU?

As our model hadn't fully converged our accuracy could be improved. Try adjusting the hyperparameters using the validation accuracy and see what test accuracy you can achieve.

## Conclusion

In this tutorial we saw an end to end example of running a GNN on IPUs for a particular task.

We built a simple GNN model based on the the `GCNConv` layer provided by the PyTorch Geometric library, trained it with PopTorch and predicted the `Cora` dataset node features.

A more in-depth overview of the data handling techniques needed for other types of graph problems can be found in the following links:

- If you are interested in "small graph" problems of the type found in healthcare and chemistry, we recommend you check out the tutorials 3 and 4 on [Small Graph Batching with Padding](../3_small_graph_batching_with_padding/3_small_graph_batching_with_padding.ipynb) and [Small Graph Batching with Packing](../4_small_graph_batching_with_packing/4_small_graph_batching_with_packing.ipynb).
- If you are looking to apply GNNs to larger graphs, similar to those found in online advertising, shopping and social networks, we recommend you to jump to our [Cluster CGN example](../../../../gnn/cluster_gcn/pytorch_geometric/node_classification_with_cluster_gcn.ipynb), which showcases how to train it for node classification using sampling. 